In [1]:
from sklearn.datasets import load_sample_image
from sklearn.feature_extraction import image
from skimage import io, color
import numpy as np
import matplotlib.pyplot as plt
import cv2
from os import listdir

In [4]:
######################################## FUNCTIONS ########################################

##### Import / Filtres

# extrait les patches d'une image
def extract_patches(path, size, patch_count) :
    img = io.imread(path)*256
    patches = image.extract_patches_2d(img, (size,size),max_patches=patch_count)
    return patches

# extrait les patches d'un dossier d'images
def folder_patches(path, size, patch_count) :
    patches = np.ndarray((0,size,size))
    for file in listdir(path) :
        new_patches = extract_patches(path+"/"+file, size, patch_count)
        patches = np.concatenate((patches, new_patches))
    return patches

#extrait les patches d'un dossier de dossiers d'images
def meta_folder_patches(path, size, patch_count,threshold) :
    for directory in listdir(path) :
        good = folder_patches(path + "/" + directory + '/Good/Prediction', size, patch_count)
        bad = folder_patches(path + "/" + directory + '/Bad/Prediction', size, patch_count)
        
        good = filter_patches(good,threshold, size)
        bad = filter_patches(bad,threshold, size)
        
        save_patches(good, './Data/Patches_train/Good/' + directory + "_good_")
        save_patches(bad, './Data/Patches_train/Bad/' + directory + "_bad_")
    
    
#Filters
        
def filter_patches(patches,seuil,size) :
    out = np.ndarray((0,size,size))
    for i in range(len(patches)) :

        if np.mean(patches[i]) > seuil :
            out = np.concatenate([out, [patches[i]]])
    
    return out

##### Affichage / Export
def patch_show(patch) :
    io.imshow(patch)
    # les images de bases sont en unsigned 16 bits donc les pixels vont de 0 --> 65535
    histogramme, classes = np.histogram(patch, bins=65536)
    print(histogramme)

    plt.figure()
    plt.xlabel("pixel value")
    plt.ylabel("pixel count")
    plt.yscale('log')
    plt.plot(classes[0:-1], histogramme)
    plt.show()
    
def patches_stats(patches) :
    print("Il y a %s patches" %(len(patches)))
    
    averages = np.mean(patches,axis=(1,2))
    #np.mean (on choisit les images qui ont une moyenne assez élevée ?)
    #np.var (on choisit les images qui ont une forte variance, donc pas que tout blanc tout noir ?)
    histogramme, classes = np.histogram(averages, bins=65536)
    #print(histogramme)

    plt.figure()
    plt.xlabel("average pixel value of a patch")
    plt.ylabel("pixel count")
    plt.yscale('log')
    plt.plot(classes[0:-1], histogramme)
    plt.show()

def save_patches(patches, path) :
    for i in range(len(patches)) :
        true_path = path+"_"+str(i)+".tif"
        cv2.imwrite(true_path,(patches[i]).astype('uint16'))



In [5]:
######################################## MAIN ########################################
PATH = "./Data/Images-Cytation/"
SIZE = 320
MAXPATCH = 10
THRESHOLD = 3

meta_folder_patches(PATH, SIZE, MAXPATCH, THRESHOLD)

KeyboardInterrupt: 